In [181]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
#from lxml import html
import pandas as pd
import re
from datetime import datetime


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)



In [182]:
def wheel_element(element, deltaY = 120, offsetX = 0, offsetY = 0):
    error = element._parent.execute_script("""
    var element = arguments[0];
    var deltaY = arguments[1];
    var box = element.getBoundingClientRect();
    var clientX = box.left + (arguments[2] || box.width / 2);
    var clientY = box.top + (arguments[3] || box.height / 2);
    var target = element.ownerDocument.elementFromPoint(clientX, clientY);

    for (var e = target; e; e = e.parentElement) {
      if (e === element) {
        target.dispatchEvent(new MouseEvent('mouseover', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new MouseEvent('mousemove', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new WheelEvent('wheel',     {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY, deltaY: deltaY}));
        return;
      }
    }    
    return "Element is not interactable";
    """, element, deltaY, offsetX, offsetY)
    if error:
        raise WebDriverException(error)

In [183]:
url = 'https://cryptopanic.com/'
driver.get(url)

res = []
for counter in range(300):
    elems = driver.find_elements(By.XPATH, "/html/body/div[1]/div[2]/div[1]/div/div[1]/div[3]/div[1]/div[contains(@class, 'news-row-link')]")
    for tmp in elems[len(res):]:
        
        # News title
        title = tmp.find_elements(By.XPATH, "./div/a[2]/span/span")[0].text
        
        # News source
        source = tmp.find_elements(By.XPATH, "./div/a[2]/span/span")[1].text
        
        # News link
        href = tmp.find_elements(By.XPATH, "./div/a")[1].get_attribute('href')
        
        # News ID
        _id = re.match('.*?(\d+).*', href).group(1)
        
        # First currency
        try:
            first_crypto = tmp.find_elements(By.XPATH, ".//div[contains(@class, 'nc-currency')]/a")[0].text
           
        except:
            first_crypto = None
        
        # Second currency
        try:
            
            second_crypto = tmp.find_elements(By.XPATH, ".//div[contains(@class, 'nc-currency')]/a")[1].text
        except:
            second_crypto = None
        
        # Third currency
        try:
            
            third_crypto = tmp.find_elements(By.XPATH, ".//div[contains(@class, 'nc-currency')]/a")[2].text
        except:
            
            third_crypto = None
        
        # Publication date
        data_time = tmp.find_elements(By.XPATH, "./div/a/span/time")[0].get_attribute('datetime')
        pub_date = re.match("\w+ (\w+ \d+ \d+).*", data_time).group(1)
        pub_date = datetime.strptime(pub_date, "%b %d %Y").strftime('%Y.%m.%d')
        
        # Positive votes
        try:
            pos_votes = tmp.find_elements(By.XPATH, ".//span[contains(@title, 'positive')]")[0].get_attribute('title')
            pos_votes = re.match('.*?(\d+).*', pos_votes).group(1)
        except:
            pos_votes = None
            
        # Negative votes
        try:
            neg_votes = tmp.find_elements(By.XPATH, ".//span[contains(@title, 'negative')]")[0].get_attribute('title')
            neg_votes = re.match('.*?(\d+).*', neg_votes).group(1)
        except:
            neg_votes = None
        
        # Important votes
        try:
            imp_votes = tmp.find_elements(By.XPATH, ".//span[contains(@title, 'important')]")[0].get_attribute('title')
            imp_votes = re.match('.*?(\d+).*', imp_votes).group(1)
        except:
            imp_votes = None
        
        # Comments votes
        try:
            com_votes = tmp.find_elements(By.XPATH, ".//span[contains(@title, 'comments')]")[0].get_attribute('title')
            com_votes = re.match('.*?(\d+).*', com_votes).group(1)
        except:
            com_votes = None
        
        # Like votes
        try:
            like_votes = tmp.find_elements(By.XPATH, ".//span[contains(@title, ' like')]")[0].get_attribute('title')
            like_votes = re.match('.*?(\d+).*', like_votes).group(1)
        except:
            like_votes = None
        
        # Dislike votes
        try:
            dislike_votes = tmp.find_elements(By.XPATH, ".//span[contains(@title, 'dis')]")[0].get_attribute('title')
            dislike_votes = re.match('.*?(\d+).*', dislike_votes).group(1)
        except:
            dislike_votes = None
        
        # Lol votes
        try:
            lol_votes = tmp.find_elements(By.XPATH, ".//span[contains(@title, 'lol')]")[0].get_attribute('title')
            lol_votes = re.match('.*?(\d+).*', lol_votes).group(1)
        except:
            lol_votes = None
            
        # Save votes
        try:
            save_votes = tmp.find_elements(By.XPATH, ".//span[contains(@title, 'save')]")[0].get_attribute('title')
            save_votes = re.match('.*?(\d+).*', save_votes).group(1)
        except:
            save_votes = None
      
        if title != '':
            res.append([_id, title, first_crypto, second_crypto, third_crypto, source, href, pub_date, pos_votes, neg_votes, imp_votes, com_votes, like_votes, dislike_votes, lol_votes, save_votes])
        else:
            break
    #time.sleep(3)        
    for i in range(len(elems) - 1, -1, -1):
        try:
            wheel_element(elems[i], 300)
            break
        except:
            pass

In [184]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [185]:
table = pd.DataFrame(res, columns=['_id', 'news_title', 'first_crypto', 'second_crypto', 'third_crypto', 'source', 'news_link', 'publication_date', 'pos_votes', 'neg_votes', 'imp_votes', 'com_votes', 'like_votes', 'dislike_votes', 'lol_votes', 'save_votes']).drop_duplicates()

In [186]:
table

,_id,news_title,first_crypto,second_crypto,third_crypto,source,news_link,publication_date,pos_votes,neg_votes,imp_votes,com_votes,like_votes,dislike_votes,lol_votes,save_votes
0,14630691,"This Week on Crypto Twitter: Elon Musk's Diamond Hands, Tom Brady Responds to Vitalik",None,None,None,decrypt.co,https://cryptopanic.com/news/14630691/This-Week-on-Crypto-Twitter-Elon-Musks-Diamond-Hands-Tom-Brady-Responds-to-Vitalik,2022.03.20,2,None,None,None,None,None,None,None
1,14630675,Inside the blockchain developers’ mind: How to avoid development hell,None,None,None,cointelegraph.com,https://cryptopanic.com/news/14630675/Inside-the-blockchain-developers-mind-How-to-avoid-development-hell,2022.03.20,None,None,1,1,1,None,None,1
2,14630674,Hoskinson Promises Solana-Like Performance for Cardano,ADA,None,None,beincrypto.com,https://cryptopanic.com/news/14630674/Hoskinson-Promises-Solana-Like-Performance-for-Cardano,2022.03.20,2,1,None,1,1,1,1,None
3,14630679,"Bitcoin (BTC) Supply on Crypto Exchanges Drops to Three-Year Lows, According to Analytics Firm",BTC,ETH,None,dailyhodl.com,https://cryptopanic.com/news/14630679/Bitcoin-BTC-Supply-on-Crypto-Exchanges-Drops-to-Three-Year-Lows-According-to-Analytics-Firm,2022.03.20,1,None,None,None,None,None,None,None
4,14630608,Ethereum's Vitalik Buterin Comes with New Sharding Proposal: What is EIP-4844?,ETH,None,None,u.today,https://cryptopanic.com/news/14630608/Ethereums-Vitalik-Buterin-Comes-with-New-Sharding-Proposal-What-is-EIP-4844,2022.03.20,2,None,1,1,2,None,1,None
5,14630564,Cardano (ADA) Ready to Surpass Ethereum's Optimism: Community,ADA,None,None,u.today,https://cryptopanic.com/news/14630564/Cardano-ADA-Ready-to-Surpass-Ethereums-Optimism-Community,2022.03.20,6,2,3,10,6,2,1,None
6,14630561,"VC Roundup: NFTs, crypto mortgages, 5G network and Web3 devs raise millions",None,None,None,cointelegraph.com,https://cryptopanic.com/news/14630561/VC-Roundup-NFTs-crypto-mortgages-5G-network-and-Web3-devs-raise-millions,2022.03.20,1,None,1,None,1,None,None,None
7,14630542,Australia gears for a major move on regulating cryptocurrencies,BTC,None,None,finbold.com,https://cryptopanic.com/news/14630542/Australia-gears-for-a-major-move-on-regulating-cryptocurrencies,2022.03.20,9,2,5,None,3,3,3,2
8,14630560,OKX Surpassed Coinbase By More Than $500 Billion in Volume in February,None,None,None,beincrypto.com,https://cryptopanic.com/news/14630560/OKX-Surpassed-Coinbase-By-More-Than-500-Billion-in-Volume-in-February,2022.03.20,1,None,1,None,None,None,None,None
9,14630480,"Cardano, its whales, and what they can do to push it to $1",ADA,None,None,,https://cryptopanic.com/news/14630480/Cardano-its-whales-and-what-they-can-do-to-push-it-to-1,2022.03.20,9,4,7,9,11,3,1,2
